In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.prompts.prompts import SimpleInputPrompt
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore

c:\Users\Yazat\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
documents = SimpleDirectoryReader("pdf").load_data()
documents

[Document(id_='fe67d52b-a01f-4083-b8ec-920edcb08484', embedding=None, metadata={'page_label': '1', 'file_name': 'rank.pdf', 'file_path': 'c:\\Users\\Yazat\\Desktop\\RAG\\pdf\\rank.pdf', 'file_type': 'application/pdf', 'file_size': 296780, 'creation_date': '2024-07-09', 'last_modified_date': '2024-07-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=" राष्ट्रीय   प्रौद्योगिकी   संस्थान   मिजोरि  \nNATIONAL INSTITUTE OF TECHNOLOGY MIZORAM  \n(An Institut ion of National Importance under Ministry of Education, Govt. of India)  \nचलतलांि , आइजोल , मिजोरि  / CHALTLANG, AIZAWL, MIZORAM  – 796012  \nPhone: 0389  - 2391236 / 2391774 / 2391699       Email: registraroffice@nitmz.ac.in        Website: www.nitmz.ac.in  \nDate:  09 /03/ 2024  \nBonafide/ Rank  Cer

In [3]:
system_prompt = """
    You are a technical assistant whose goal is to answer 
    questions based on the instructions 
    and the context provided. 
    If you do not have an answer from 
    the provided information say so.
    """
    
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [4]:
llm = Ollama(model="llama3:8b", request_timeout = 20000)
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

Settings.llm = llm
Settings.embed_model = embed_model


In [5]:
username = "neo4j"
password = "12345678"
url = "neo4j://localhost:7687"
database = "neo4j"

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

index = KnowledgeGraphIndex.from_documents(
    documents, 
    storage_context=storage_context,
    llm=llm,
    embed_model=embed_model,
    show_progress=True,
    max_triplets_per_chunk=3,
    include_embeddings=True
)

print("KnowledgeGraphIndex created successfully")

Processing nodes: 100%|██████████| 1/1 [10:38<00:00, 638.33s/it]


KnowledgeGraphIndex created successfully


In [6]:
query_engine = index.as_query_engine(
    include_text=True, response_mode="tree_summarize"
)

response = query_engine.query(
    "Tell me about Yazat"
)

In [7]:
from IPython.display import Markdown, display

display(Markdown(f"<b>{response}</b>"))

<b>A bonafide student of National Institute of Technology Mizoram, studying in the Department of Electronics and Communication Engineering, currently in his third year of Bachelor of Technology (B.Tech) for the academic calendar 2023-2024. He has successfully completed the 5th semester examination with a CGPA of 9.48 on a 10-point scale, ranking 1st out of 34 students in his branch, which represents top 20% in the class.</b>

In [8]:
response = query_engine.query(
    "What is his CGPA?"
)

display(Markdown(f"<b>{response}</b>"))

<b>His CGPA is 9.48 on a 10-points scale.</b>